# Requirements

In [0]:
!pip install larq tensorflow-datasets

In [0]:
import tensorflow as tf
from tensorflow import keras
import larq as lq
import numpy as np
import tensorflow_datasets as tfds
from keras.utils import np_utils


# Get SVHN dataset from tf datasets

In [0]:
train_data = tfds.load(name="svhn_cropped", split="train", batch_size=-1 ) 
test_data = tfds.load(name="svhn_cropped", split="test", batch_size=-1)
val_data = tfds.load(name="svhn_cropped", split = "extra", batch_size=-1)

In [0]:
#convert to numpy
train = tfds.as_numpy(train_data)
test = tfds.as_numpy(test_data)
val = tfds.as_numpy(val_data)

In [0]:
#Split x and y
x_train, y_train = train["image"], train["label"]
x_test, y_test = test["image"], test["label"]
x_val, y_val = val["image"], val["label"]

In [0]:
#Reduce val set
x_val, y_val = x_val[:30000,:,:,:], y_val[:30000]

In [0]:
print("Training samples: {} \nTest samples: {} \nVal samples: {}".format(len(x_train),len(x_test),len(x_val)))

In [0]:
#Data Augmentation
def resize_and_flip(image, labels, training):
    #Normalize
    image = tf.cast(image, tf.float32) / (255./2.) - 1. 
    if training:
        #Padding
        image = tf.image.resize_with_crop_or_pad(image, 40, 40)
        #crop
        image = tf.image.random_crop(image, [32, 32, 3])
        #flip
        image = tf.image.random_flip_left_right(image)
    return image, labels


In [0]:
#Dataset creation (batch generation)
def create_dataset(images,labels, batch_size, training):
    labels = tf.one_hot(np.squeeze(labels), 10)
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.repeat()
    if training:
        dataset = dataset.shuffle(1000)
    dataset = dataset.map(lambda x, y: resize_and_flip(x, y, training))
    dataset = dataset.batch(batch_size)
    return dataset

In [0]:
batch_size = 256
train_dataset = create_dataset(x_train,y_train, batch_size, True)
val_dataset = create_dataset(x_val,y_val, batch_size, False)

# Model Creation

In [0]:
#Derived from example at https://larq.dev/

def create_model(model_params):
    model = tf.keras.models.Sequential([
        lq.layers.QuantConv2D(128, 3,
                              kernel_quantizer="ste_sign",
                              kernel_constraint="weight_clip",
                              use_bias=False,
                              input_shape=(32, 32, 3)),
        tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

        lq.layers.QuantConv2D(128, 3, padding="same", **model_params),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

        lq.layers.QuantConv2D(256, 3, padding="same", **model_params),
        tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

        lq.layers.QuantConv2D(256, 3, padding="same", **model_params),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

        lq.layers.QuantConv2D(512, 3, padding="same", **model_params),
        tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

        lq.layers.QuantConv2D(512, 3, padding="same", **model_params),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),
        tf.keras.layers.Flatten(),

        lq.layers.QuantDense(1024, **model_params),
        tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

        lq.layers.QuantDense(1024, **model_params),
        tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

        lq.layers.QuantDense(10, **model_params),
        tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),
        tf.keras.layers.Activation("softmax")
    ])
    
    return model

In [0]:
model_params = dict(input_quantizer="ste_sign",
              kernel_quantizer= tf.keras.layers.Activation("linear"),
              kernel_constraint= None,
              use_bias=False)

# Optimizer declaration

In [0]:
#Return suitable optimizer bop or adam
def optimizer_fn(bop=True):
    if bop:
        initial_lr = 1e-2
    else:
        initial_lr = 1e-3
    threshold_val = 1e-8
    gamma_val = 1e-4
    gamma_decay = 0.1
    decay_step = int((50000 / 50) * 100)
    adam = tf.keras.optimizers.Adam(lr=initial_lr)
    
    if bop: 
        optimizer=lq.optimizers.Bop(fp_optimizer=adam, threshold= threshold_val, gamma=tf.keras.optimizers.schedules.ExponentialDecay(
            gamma_val, decay_step, gamma_decay, staircase=True))
        return optimizer
    
    else:
        return adam
   


In [0]:
def lr_schedule(epoch):
    return 1e-3 * 0.1 ** (epoch // 100)


# Train model and save 

In [0]:
def train_model(bop=True):
    model = create_model(model_params)
    model.compile(
        optimizer=optimizer_fn(bop),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    logs = keras.callbacks.CSVLogger('svhn.log')
    callbacks = [logs]
    if not bop:
        scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)
        callbacks.append(scheduler)
    
    trained_model = model.fit(
    train_dataset,
    epochs=300,
    steps_per_epoch= len(x_train) // batch_size,
    validation_data=val_dataset,
    validation_steps= len(x_val) // batch_size,
    verbose=1,
    callbacks=callbacks
    )
    if bop:
        model.save('svhn_bop.h5')
    else:
        model.save('svhn_baseline.h5')

    

In [0]:
def test_model(bop=True):
    model = create_model(model_params)
    model.compile(
        optimizer=optimizer_fn(bop),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    if bop:
        model.load_weights('svhn_bop.h5')
    else:
        model.load_weights('svhn_baseline.h5')
    y_new = np_utils.to_categorical(y_test)
    scores = model.evaluate(x_test, y_new, verbose=1)
    print("Accuracy of model is {}".format(scores[1]*100.0))

In [0]:
# For baseline set bop to false
train_model(bop=True)
test_model(bop=True)